# Настройка доступов и подгрузка библиотек

In [ ]:
meme_pattern = 'change my mind'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -s "/content/drive/My Drive" /gdrive

In [ ]:
%tensorflow_version 1.x
!pip install git+https://github.com/SerSmith/gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
  Cloning https://github.com/SerSmith/gpt-2-simple to /tmp/pip-req-build-ldf8mgw4
  Running command git clone -q https://github.com/SerSmith/gpt-2-simple /tmp/pip-req-build-ldf8mgw4
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=25644 sha256=0ac6ce01277f93d002a1f81bd10684958f5fcf564405e66923435c5bb21c69cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-7tahxdm6/wheels/05/e5/9d/9b6ebf15d32922867291a66ba17b8610a5621e6cd1a399fba5
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Подготовка датасета для дообучения

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import os
import pickle
import nltk
import shutil
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

path_raw_data = '/gdrive/MADE_PROJECT/raw_data/memes_all_eng_v2.csv'
path_folder_gdisk = '/content/drive/My Drive'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv(path_raw_data)
df['template_name_prep'].value_counts()

batman slapping robin    18241
x x everywhere           16677
waiting skeleton         12314
change my mind           12169
disaster girl             9418
Name: template_name_prep, dtype: int64

In [ ]:
def prepare_text(text):
  text = str(text).strip().upper()
  text = re.sub(r"\s", " ", text)
  text = re.sub(r"\n", " ", text)
  text = re.sub(r"&", "and", text)
  text = re.sub(r"|", "", text)
  return text

def prepare_train(data, template, dest_path):
  out = df[df['template_name_prep'] == template]['text'].drop_duplicates().reset_index(drop=True).apply(prepare_text)
  print(out.shape)
  out.to_csv(dest_path, index=False, header=False)
  return out

q = prepare_train(df, meme_pattern, os.path.join(path_folder_gdisk, f'{meme_pattern}.csv'))
table = os.path.join(path_folder_gdisk, f'{meme_pattern}.csv')
!cat "$table" | head -n 10

(12034,)
EVERY DAY WITH WENDY IS A WEN-DAY||
CHELKAT ARE ONLY FRIENDS WITH MEGAN SO THEY HAVE SOMEONE TO BE MEAN TO||
WORKING FROM HOME ISN'T AS AWESOME AS WE ANTICIPATED||
KEVIN SHOULD BE ALLOWED BACK IN||
"CALLING ME ""PARANOID"" DOESN'T PROVE YOU'RE NOT OUT TO GET ME.||"
CHARLIE BAKER IS THE BIGGEST PUSSY IN POLITICS.||
WASH YOUR HANDS OR END UP LIKE SM||
CAN ANYBODY|FEEL ME OUT THERE|
"I DON'T NEED HELP, I'M ENOUGH AS IS|BEOWULF|"
COVID-19 IS A GENETICALLY ENGINEERED BIOWEAPON||


Запуск дообучения

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
!nvidia-smi

Tue Nov 24 15:50:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.mount_gdrive()
gpt2.download_gpt2(model_name="124M")
gpt2.copy_file_from_gdrive(f'{meme_pattern}.csv')

history = gpt2.finetune(sess,
        f'{meme_pattern}.csv',
        steps=2500,
        model_name='124M',
        model_dir='models',
        combine=50000,
        batch_size=1,
        learning_rate=0.0001,
        accumulate_gradients=5,
        restore_from='latest',
        run_name=meme_pattern,
        checkpoint_dir='checkpoint',
        sample_every=50,
        sample_length=100,
        sample_num=500,
        multi_gpu=False,
        save_every=1000,
        print_every=50)

Fetching checkpoint: 1.05Mit [00:00, 379Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 132Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 397Mit/s]                                                    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



Fetching model.ckpt.data-00000-of-00001: 498Mit [00:01, 275Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 272Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 172Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 138Mit/s]                                                       


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 72.82it/s]

...

[1500 | 19176.03] loss=0.07 avg=0.58

|STEVEN WEISSENBERG TOLLOW UNHOLY PAPER||<|endoftext|>
<|startoftext|>FREDDIE MERCURY IS THE OPPOSITE OF DOG EAT PSYCHOTROPIC EXPERT||<|endoftext|>
<|startoftext|>ONE BY ED SHEERAN IS A GREAT SONG||<|endoftext|>
<|


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/scipy/sparse/base.py:599: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


[1550 | 19815.72] loss=0.05 avg=0.56
KG IS JUST RACIST<|endoftext|>
<|startoftext|>PEOPLE WHO ARGUE JUST TO ARGUE ARE UNAGGRY<|endoftext|>
<|startoftext|>DOLPHINS ARE JUST UNICORNS<|endoftext|>
<|startoftext|>UNDERTALE IS CRAP. CUSTOM ZONE<|endoftext|>
<|


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[1600 | 20449.31] loss=0.05 avg=0.55


In [ ]:
with open(f'/gdrive/{meme_pattern}_history.pickle', 'wb') as f:
  pickle.dump(history, f)

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=meme_pattern)

# Генерация текста

In [ ]:
# sess = gpt2.start_tf_sess()
# gpt2.load_gpt2(sess, run_name='disaster girl 12 11')
# gpt2.generate(sess,
#               length=100,
#               temperature=0.7,
#               prefix="Change my mind",
#               nsamples=5,
#               batch_size=5,
#               truncate='EOF'
#               )